# Deep Residual Learning for Image Recognition

### Imports

In [ ]:
import tensorflow as tf

### Build blocks

#### Identity Block

<div align="center">

  <img alt="Identity Block" src="assets/identity_block_skip3.png" width=800 height=250/>
  <br/>
  <figcaption>Figure 1: Identity Block. Skip connection: skips over 3 layers</figcaption>

</div>

In [14]:
def identity_block(inputs, f, filters):
    """
    Identity block that skips over 3 layers

    Args:
        inputs (tensor): input tensor of shape (b, H, W, C)
        f (int): the shape of the middle Conv's window for the main path
        filters (List[int]): number of filters

    Returns:
        output of the identity block, tensor of shape (b, H, W, C)
    """

    # retrieve filters
    f1, f2, f3 = filters

    # save input value
    input_shortcut = inputs

    # 1st component of the main path
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=1, strides=(1, 1), padding="valid")(inputs)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)


    # 2nd component
    x = tf.keras.layers.Conv2D(filters=f2, kernel_size=f, strides=(1, 1), padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)


    # 3rd component
    x = tf.keras.layers.Conv2D(filters=f3, kernel_size=1, strides=(1, 1), padding="valid")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)


    # final step: add shortcut value to main path, and pass it through a ReLU
    x = tf.keras.layers.Add()([x, input_shortcut])
    x = tf.keras.layers.Activation("relu")(x)

    return x

#### Convolutional Block


<div align="center">

  <img alt="Convolutional Block" src="assets/convolutional_block_skip3.png" width=800 height=250/>
  <br/>
  <figcaption>Figure 2: Convolutional Block</figcaption>

</div>

In [15]:
def convolutional_block(inputs, f, filters, s=2):
    """
    Convolutional block that skips over 3 layers

    Args:
        inputs (tensor): input tensor of shape (b, H, W, C)
        f (int): the shape of the middle Conv's window for the main path
        filters (List[int]): number of filters
        s (int): stride value

    Returns:
        output of the convolutional block, tensor of shape (b, H, W, C)
    """

    # retrieve filters
    f1, f2, f3 = filters

    # save input value
    input_shortcut = inputs

    # 1st component of main path
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=1, strides=(s, s), padding="valid")(inputs)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)

    # 2nd component
    x = tf.keras.layers.Conv2D(filters=f2, kernel_size=f, strides=1, padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)

    # 3rd component
    x = tf.keras.layers.Conv2D(filters=f3, kernel_size=1, strides=1, padding="valid")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    # shortcut path
    input_shortcut = tf.keras.layers.Conv2D(
        filters=f3, kernel_size=1, strides=(s, s), padding="valid"
    )(input_shortcut)
    input_shortcut = tf.keras.layers.BatchNormalization(axis=3)(input_shortcut)

    # final step: add shortcut value to main path, and pass it through a ReLU
    x = tf.keras.layers.Add()([x, input_shortcut])
    x = tf.keras.layers.Activation("relu")(x)

    return x

### ResNet Model

<div align="center">

  <img alt="ResNet-50" src="assets/ResNet_50.png" width=1000 height=250/>
  <br/>
  <figcaption>Figure 3: ResNet-50</figcaption>

</div>

In [20]:
def resnet50(input_shape=(64, 64, 3), n_classes=6, training=False):
    """
    Stage-wise implementation of ResNet50 architecture:
    Conv2D -> BatchNorm -> ReLU -> MaxPool -> ConvBlock -> IdBlock * 2 -> ConvBlock -> IdBlock * 3
    -> ConvBlock -> IdBlock * 5 -> ConvBlock -> IdBlock * 2 -> AvgPool -> Flatten -> Dense

    Args:
        input_shape (tuple): shape of the images
        n_classes (int): number of classes
        training (bool): training or not

    Returns:
        keras model instance
    """

    inputs = tf.keras.layers.Input(input_shape)


    # zero-padding
    x = tf.keras.layers.ZeroPadding2D((3, 3))(inputs)

    # stage 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # stage 2
    x = convolutional_block(x, f=3, filters=[64, 64, 256], s=1)
    x = identity_block(x, f=3, filters=[64, 64, 256])
    x = identity_block(x, f=3, filters=[64, 64, 256])

    # stage 3
    x = convolutional_block(x, f=3, filters=[128, 128, 512], s=2)
    x = identity_block(x, f=3, filters=[128, 128, 512])
    x = identity_block(x, f=3, filters=[128, 128, 512])
    x = identity_block(x, f=3, filters=[128, 128, 512])

    # stage 4
    x = convolutional_block(x, f=3, filters=[256, 256, 1024], s=2)
    x = identity_block(x, f=3, filters=[256, 256, 1024])
    x = identity_block(x, f=3, filters=[256, 256, 1024])
    x = identity_block(x, f=3, filters=[256, 256, 1024])
    x = identity_block(x, f=3, filters=[256, 256, 1024])
    x = identity_block(x, f=3, filters=[256, 256, 1024])

    # stage 5
    x = convolutional_block(x, f=3, filters=[512, 512, 2048], s=2)
    x = identity_block(x, f=3, filters=[512, 512, 2048])
    x = identity_block(x, f=3, filters=[512, 512, 2048])

    # avg pool
    x = tf.keras.layers.AveragePooling2D((2, 2))(x)

    # output layer
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(n_classes, activation="softmax")(x)


    model = tf.keras.Model(inputs=inputs, outputs=x, name="ResNet-50")

    return model

In [21]:
model = resnet50(input_shape=(64, 64, 3), n_classes=6)
model.summary()

Model: "ResNet-50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_5    │ (None, 70, 70, 3) │          0 │ input_layer_5[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_122 (Conv2D) │ (None, 32, 32,    │      9,472 │ zero_padding2d_5… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_122[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_110      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 15, 15,    │          0 │ activation_110[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_123 (Conv2D) │ (None, 15, 15,    │      4,160 │ max_pooling2d_5[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_123[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_111      │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_124 (Conv2D) │ (None, 15, 15,    │     36,928 │ activation_111[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_124[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_112      │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_125 (Conv2D) │ (None, 15, 15,    │     16,640 │ activation_112[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_126 (Conv2D) │ (None, 15, 15,    │     16,640 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_125[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_126[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_35 (Add)        │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 23,600,006 (90.03 MB)

 Trainable params: 23,546,886 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

### Compile the Model

In [22]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00015),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)